In [1]:
import pandas3js as pjs
print(pjs.__version__)
import jsonextended as ejson
print(ejson.__version__)
import pandas as pd
import numpy as np
import pythreejs as tjs

0.0.3
0.1.3.3


In [2]:
data = ejson.json_to_dict(ejson.get_test_path())
ejson.dict_pprint(data, depth=3)

dir1: 
  dir1_1: 
    file1_1: {...}
  file1: 
    initial: {...}
    meta: {...}
    optimised: {...}
    units: {...}
  file2: 
    initial: {...}
    meta: {...}
    optimisation: {...}
    optimised: {...}
    units: {...}
dir2: 
  file1: 
    initial: {...}
    meta: {...}
    optimisation: {...}
    optimised: {...}
    units: {...}
dir3: 


In [3]:
data = ejson.dict_remove_paths(data, ['units'])
energies = ejson.dict_filter_keys(data, ['energy'])
pd.Series(ejson.dict_flatten(energies)).describe()

count       89.000000
mean    -24062.286741
std          0.012361
min     -24062.293964
25%     -24062.293727
50%     -24062.293491
75%     -24062.278072
max     -24062.207939
dtype: float64

In [4]:
energies = ejson.units.apply_unitschema(energies, {'energy':'eV'})
energies = ejson.units.apply_unitschema(energies, {'energy':'kcal'},as_quantity=False)
pd.Series(ejson.dict_flatten(energies)).describe()

count    8.900000e+01
mean    -9.214157e-19
std      4.733481e-25
min     -9.214159e-19
25%     -9.214159e-19
50%     -9.214159e-19
75%     -9.214153e-19
max     -9.214126e-19
dtype: float64

In [118]:
data = ejson.json_to_dict(ejson.get_test_path())
optimisation = ejson.dict_multiindex(data,['dir1','file2','optimisation','steps'])
optsteps = sorted(optimisation.keys(), key=ejson.core._natural_keys)
print(optsteps)

['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90']


In [120]:
ejson.dict_pprint(optimisation['2'],
                  depth=None,no_values=True)

crystallographic: 
  geometry: 
    assym:         
    atomic_number: 
    id:            
    label:         
    x/a:           
    y/b:           
    z/c:           
  lattice_parameters: 
    a:     
    alpha: 
    b:     
    beta:  
    c:     
    gamma: 
  volume: 
energy: 
primitive: 
  density: 
  geometry: {...}
  lattice_parameters: {...}
  volume:  


In [7]:
ejson.dict_to_html(optimisation['2'])

In [ ]:
import ipywidgets as widgets

def _create_callback(renderer, data, select, ddown,
                     change_func, gcollect, all_options,
                     otype_column):
    def handle_ddown(change):
        with renderer.hold_trait_notifications():
            all_options[ddown.description] = ddown.value
            geometry = change_func(data[select.value], 
                                   all_options)
            gcollect.change_by_df(geometry,otype_column=otype_column,
                        otype_default='pandas3js.Sphere',
                                 remove_missing=True)        
    return handle_ddown

def create_config_gui(data, change_func, 
                      add_objects=True, add_labels=True,
                      view=(10,-10,-10,10),near=-10,
                      add_options=None,
                      otype_column=None):
    """ creates simple gui to handle geometric configuration changes

    Properties
    ----------
    data : dict
        {config_name:cdata} pairs
    change_func : function
        change_func(cdata,options_dict) -> pandas.DataFrame
        of geometric objects and traits
    add_objects : bool
        add objects to scene
    add_labels : bool
        add object labels to scene
    view : tuple
        initial view extents (top,bottom,left,right)
    near : int
        camera distance from origin    
    add_options : None or dict
        additional option lists, to create dropdown boxes 
        with callbacks to change_func as options dict
    otype_column : str
        column name for object type 
        in dataframe from change_func
    
    Returns
    -------
    gui : widgets.Box
        containing rendered scene and option widgets
    gcollect : pandas3js.GeometricCollection
        the collection of current geometric objects
        
    Examples
    --------

    >>> import pandas as pd
    >>> data = {'1':{'id':[0],'x':[0],'y':[0],'z':[0],
    ...              'c1':'red','c2':'blue'},
    ...         '2':{'id':[0],'x':[1],'y':[2],'z':[3],
    ...              'c1':'red','c2':'blue'}}
    ...
    >>> def change_func(cdata,options):
    ...     indf = pd.DataFrame(cdata)
    ...     ctype = options.get('color','c1')
    ...     indf['color'] = indf[ctype]
    ...     indf['label'] = 'myobject'
    ...     return indf[['id','x','y','z','color','label']]
    ...
    >>> gui, collect = create_config_gui(data,change_func,
    ...                     add_options={'color':['c1','c2']})
    ...
    >>> [type(c) for c in gui.children]
    [ipywidgets.widgets.widget_selectioncontainer.Tab,
    pythreejs.pythreejs.Renderer]
    >>> collect.trait_df().loc[0]
    color                                              red
    id                                                   0
    label                                                -
    label_color                                        red
    label_transparency                                   1
    label_visible                                    False
    otype                 pandas3js.models.idobject.Sphere
    radius                                               1
    transparency                                         1
    visible                                           True
    x                                                    1
    y                                                    2
    z                                                    3
    Name: 0, dtype: object    
    >>> config_select = gui.children[0].children[0].children[0].children[0]
    >>> type(config)
    ipywidgets.widgets.widget_selection.SelectionSlider
    >>> config_select.value = '1'
    >>> collect.trait_df().loc[0]
    color                                              red
    id                                                   0
    label                                                -
    label_color                                        red
    label_transparency                                   1
    label_visible                                    False
    otype                 pandas3js.models.idobject.Sphere
    radius                                               1
    transparency                                         1
    visible                                           True
    x                                                    0
    y                                                    0
    z                                                    0
    Name: 0, dtype: object
    >>> color_select = gui.children[0].children[1].children[0]
    >>> type(color_select)
    ipywidgets.widgets.widget_selection.Dropdown
    >>> color_select.value = 'c2'
    >>> collect.trait_df().loc[0]
    color                                             blue
    id                                                   0
    label                                                -
    label_color                                        red
    label_transparency                                   1
    label_visible                                    False
    otype                 pandas3js.models.idobject.Sphere
    radius                                               1
    transparency                                         1
    visible                                           True
    x                                                    0
    y                                                    0
    z                                                    0
    Name: 0, dtype: object
    
    """
    controls = []
    all_options = {}
    
    # sort number strings correctly
    dkeys = sorted(data.keys(), key=pjs.utils.natural_keys)

    gcollect = pjs.GeometricCollection()
    scene = pjs.create_js_scene_view(gcollect,
                    add_objects=add_objects,add_labels=add_labels)
    camera, renderer = pjs.create_jsrenderer(scene,view=view,near=near)
    
    # a slider for selecting the configuration
    select = widgets.SelectionSlider(description='Configuration:',
                options=dkeys, continuous_update=False)
    def handle_slider(change):
        with renderer.hold_trait_notifications():
            geometry = change_func(data[change.new],all_options)
            gcollect.change_by_df(geometry,otype_column=otype_column,
                        otype_default='pandas3js.Sphere',
                        remove_missing=True)        
    select.observe(handle_slider,names='value')
    select.value = dkeys[-1]
    controls.append(select)

    # a check box for showing labels
    if add_labels:    
        toggle=widgets.Checkbox(
        value=False,
        description='View Label:')
        def handle_toggle(change):
            for obj in gcollect.idobjects:
                obj.label_visible = change.new
        toggle.observe(handle_toggle,names='value')
        controls.append(toggle)
    
    # a zoom slider
    top,bottom,left,right = view
    axiszoom = widgets.FloatRangeSlider(
        value=[left,right],
        min=left,
        max=right,
        step=abs(right-left)/100,
        description='axis range',
        readout_format='.1f',
        continuous_update=True,)
    def handle_axiszoom(change):
        camera.left =  camera.bottom = change.new[0]
        camera.right = camera.top = change.new[1]
    axiszoom.observe(handle_axiszoom,names='value')
    
    # add additional options
    opt_dropdowns = []
    add_options = {} if add_options is None else add_options
    for label, options in add_options.items():
        ddown = widgets.Dropdown(options=options,
                        description=label,value=options[-1])
        handle = _create_callback(renderer, data, select, ddown,
                                 change_func, gcollect,all_options,
                                 otype_column)
        ddown.observe(handle, names='value')
        opt_dropdowns.append(ddown)
        ddown.value = options[0]
    
    if opt_dropdowns:
        options = widgets.Tab(
            children=[widgets.VBox([widgets.HBox(controls),
                                   axiszoom]), 
                      widgets.VBox(opt_dropdowns)])
        options.set_title(0, 'Main Controls')
        options.set_title(1, 'Other Options')
    else:
        options = widgets.VBox([widgets.HBox(controls),
                               axiszoom])
    
    return widgets.VBox([options,
                         renderer]), gcollect

In [167]:
gui.close()

In [112]:
from scipy.spatial import cKDTree
cKDTree.query_pairs

def format_label(s):
    s = ''.join([i for i in s if not i.isdigit()])
    return s.lower().capitalize()

def change_crystal_opt(data, options):
    
    geotype = 'primitive'#'crystallographic'

    # lattice vectors
    ldict = data[geotype]['lattice_parameters']
    a_vec, b_vec, c_vec = pjs.atom.vectors_from_params(
        *[ldict[s] for s in ('a','b','c','alpha','beta','gamma')])
    
    gdict = data[geotype]['geometry']
    df = pd.DataFrame(gdict)
        
#     xyz = (df['x/a'].apply(lambda x:x*a_vec) + 
#            df['y/b'].apply(lambda y:y*b_vec) + 
#            df['z/c'].apply(lambda z:z*c_vec))
#     xyz = np.array(xyz.values.tolist())
#     df['x'] = xyz[:,0]
#     df['y'] = xyz[:,1]
#     df['z'] = xyz[:,2]
        
    df['x'] = df['x/a']*ldict['a']
    df['y'] = df['y/b']*ldict['b']
    df['z'] = df['z/c']*ldict['c']
    
    df.label = df.label.apply(format_label)
    
    atom_df = pjs.atom.atomic_data()
    df['color'] = df['atomic_number'].map(lambda n: atom_df.loc[n].color)
    df['label_color'] = df['color']
    df['radius'] = df['atomic_number'].map(lambda n: atom_df.loc[n].RCov)
    df['transparency'] = 1
    
    df = df[['id','x','y','z',
             'label','label_color',
             'color','radius']]
    df['transparency'] = 1
    df['otype'] = 'pandas3js.Sphere'
    
    # sub-lattices
    if options.get('show coordination', False):
        for aname, anumber in (['S',16],):#,['Fe',26]):
            r_array = df[df.label==aname][['x','y','z']].values    
            ck = cKDTree(r_array)
            pairs = ck.query_pairs(4)
            for i,j in pairs:
                series = pd.Series({
                    'start':tuple(r_array[i]),'end':tuple(r_array[j]),
                    'color':atom_df.loc[anumber].color,'otype':'pandas3js.Line',
                    'id':df['id'].max()+1,'linewidth':3})
                df = df.append(series,ignore_index=True,)
    
#     series = pd.Series({'a':tuple(a_vec),'b':tuple(b_vec),'c':tuple(c_vec),
#                        'otype':'pandas3js.WireBox','id':df['id'].max()+1,'color':'black'})    
    #df = df.append(series,ignore_index=True,)

    return df

In [123]:
data = ejson.json_to_dict(ejson.get_test_path(),['dir1','file2','optimisation','steps'])
gui, geometry = create_config_gui(data, change_crystal_opt,near=-15,
                                  add_labels=True,view=(10, -10, -10, 10),
                                  otype_column='otype',
                                  add_options={'show coordination':[False, True],
                                              })
gui

In [51]:
geometry.trait_df().head()

,color,end,id,label,label_color,label_transparency,label_visible,linewidth,otype,radius,start,transparency,visible,x,y,z
0,"(0.7, 0.7, 0.0)",NaN,1,S,"(0.7, 0.7, 0.0)",1.0,False,NaN,pandas3js.models.idobject.Sphere,1.05,NaN,1.0,True,-1.755404,-0.091516,4.964415
1,"(0.7, 0.7, 0.0)",NaN,2,S,"(0.7, 0.7, 0.0)",1.0,False,NaN,pandas3js.models.idobject.Sphere,1.05,NaN,1.0,True,-0.091516,-1.755404,1.515249
2,"(0.7, 0.7, 0.0)",NaN,3,S,"(0.7, 0.7, 0.0)",1.0,False,NaN,pandas3js.models.idobject.Sphere,1.05,NaN,1.0,True,1.755404,0.091516,-4.964415
3,"(0.7, 0.7, 0.0)",NaN,4,S,"(0.7, 0.7, 0.0)",1.0,False,NaN,pandas3js.models.idobject.Sphere,1.05,NaN,1.0,True,0.091516,1.755404,-1.515249
4,"(0.7, 0.7, 0.0)",NaN,5,S,"(0.7, 0.7, 0.0)",1.0,False,NaN,pandas3js.models.idobject.Sphere,1.05,NaN,1.0,True,1.748257,-3.488464,-4.866776


# Example

In [35]:
import pandas as pd
import pandas3js as pjs

data = {'1':{'id':[0,1,2],
             'geometry':{
                 'x':[0,1,2],'y':[0,1,0],'z':[0,0,1]},
              'c1':'blue','c2':'red'},
        '2':{'id':[0,1,2],
             'geometry':{
                 'x':[0,.5,1],'y':[0,0,1],'z':[0,0.5,1]},
              'c1':'blue','c2':'red'}}

def change_func(cdata,options):
    df = pd.DataFrame(cdata['geometry'])
    ctype = options.get('color','c1')
    df['id'] = cdata['id']
    df['color'] = cdata[ctype]
    df['radius'] = 0.5
    df['label'] = 'A'
    df['otype'] = 'pandas3js.Sphere'
    return df

gui, gcollect = pjs.create_config_gui(data,change_func,
                    height=200,width=200,
                    view=(3,-3,-3,3),otype_column='otype',
                    add_options={'color':['c1','c2']})

In [36]:
gui

In [37]:
trait_df = gcollect.trait_df()
trait_df

,color,id,label,label_color,label_transparency,label_visible,otype,radius,transparency,visible,x,y,z
0,red,0,A,red,1.0,True,pandas3js.models.idobject.Sphere,0.5,1.0,True,0.0,0.0,0.0
1,red,1,A,red,1.0,True,pandas3js.models.idobject.Sphere,0.5,1.0,True,1.0,1.0,0.0
2,red,2,A,red,1.0,True,pandas3js.models.idobject.Sphere,0.5,1.0,True,2.0,0.0,1.0


In [38]:
trait_df.transparency = 0.1
gcollect.change_by_df(trait_df,
                      otype_column='otype')